<a href="https://colab.research.google.com/github/yue-sun/generative-art/blob/d1_geom_fractals/01_monday/01_geometric_fractals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Geometric Fractals

Geometric fractals are formed by repeating a pattern or action at smaller and smaller scales. Fractals can be observed in many fascinating natural examples, from branching river networks, blood vessels, and lightning, to trees, broccoli, and ice crystals! According to Benoit Mandelbrot:

> "Bottomless wonders spring from simple rules, which are repeated without end."

The repetition of simple rules is the key principle behind generating geometric fractals that we will look at in this session.

## Turtle Graphics

In order to visualize our mathematical art, we will make use of [ColabTurtle](https://github.com/tolgaatam/ColabTurtle). Let's start by installing the necessary package:

In [ ]:
print('installing ColabTurtle...')
!pip3 install ColabTurtle > /dev/null
print('done!')

We'll set up our window size, a small palette of colors, and linewidth (which you are welcome to change!) The full list of available colors can be [found here (line 25)](https://github.com/tolgaatam/ColabTurtle/blob/master/ColabTurtle/Turtle.py).

In [ ]:
import numpy as np
from ColabTurtle.Turtle import *

# define a small palette of colors to use
palette = {'beige':'linen', 'red':'darksalmon',
           'yellow': 'burlywood', 'green':'olivedrab',
           'blue':'cadetblue'}

W = 600
H = 400
initializeTurtle(initial_window_size=(W,H))
bgcolor(palette['beige']) # background color
color(palette['green'])  # turtle color
width(3)

The following are a few basic functions to know when using ColabTurtle:

- `clear()` : Clear the screen of all drawings.
- `penup()/pendown()` : Lift/lower the pen for drawing.
- `forward(units)` : Advance turtle forward by the given number of pixel units. When the pen is down, this draws a line.
- `left(degrees)/right(degrees)` : Rotate the turtle's heading left or right by the specified degrees.
- `hideturtle()/showturtle()` : Hide or display the turtle icon on the screen.
- `speed(value)` : Set the turtle draw speed, an integer on $[1,13]$.
- `setx(x)/sety(y)`: Set the x and y position of the turtle (in pixels).
- `getx()/gety()`: Returns the x and y position of the turtle.
- `setheading(h)/getheading()`: Set or return the heading of the turtle. 0 degrees points due East, 90 degrees points due South.

Let's practice a simple drawing:

In [ ]:
# clear the screen
clear()

# set the speed
speed(5)

# set position and heading of the turtle with pen up
showturtle()
penup()
L = 200
setx(200); sety(100)
setheading(0)
pendown()

# draw a line
color(palette['red'])
forward(L)

# draw another line in a new color
right(90)
color(palette['yellow'])
forward(L)

# repeat!
right(90)
color(palette['green'])
forward(L)

# repeat!
right(90)
color(palette['blue'])
forward(L)

# hide turtle at the end
hideturtle()

## Lindenmayer Systems (L-Systems)

Lindenmayer systems are a type of "grammar", or writing rules, developed in 1968 by botanist Aristid Lindenmayer to model the growth and development of plants. In a nutshell, L-systems are a compact way of representing an iterative geometric structure using symbols. When we attribute a meaning to each letter or symbol, such as whether to draw a line or rotate our heading, we can generate beautiful geometric fractals.

L-systems are defined by three components:
> 1. An **alphabet**, or set of symbols to use
2. An **axiom**, or initiator string of symbols
3. A set of **rules** specifying how each symbol in the alphabet evolves in the next generation
> 
>Also, there can be two types of symbols in the alphabet:
> >1. **variables**, which are replaced in subsequent generations
2. **constants**, which are not replaced

Consider the following L-system:
1. Alphabet: `F` (variable) `+`, `-` (constants)
2. Axiom: `F`
3. Rule: `F --> F+F--F+F`

What do the first three generations of this L-system look like?

- Generation 0: `F` (axiom)
- Generation 1: <font color=red>`F+F--F+F`</font>
- Generation 2: <font color=orange>`F+F--F+F`</font><font color=red>`+`</font><font color=green>`F+F--F+F`</font><font color=red>`--`</font><font color=blue>`F+F--F+F`</font><font color=red>`+`</font><font color=purple>`F+F--F+F`</font>

We can start to see that generating subsequent iterations lends itself to a recursive algorithm. Now let's attribute meaning to each symbol:
- `F` : advance forward and draw a line
- `+` : turn left 60 degrees
- `-` : turn right 60 degrees

When interpreted in this way, this fairly simple L-system in fact describes the generation rule for the Koch curve! Let's see this in action:

In [ ]:
#function to create koch curve
def koch_curve(depth, headings, length, symbol='F'):
    if depth == 0:
        forward(length)
        headings += [heading()]
        return
    else:
        koch_curve(depth-1, headings, length/3., 'F') # F

        left(60) # +
        koch_curve(depth-1, headings, length/3., 'F') # F

        right(120) # --
        koch_curve(depth-1, headings, length/3., 'F') # F

        left(60) # +
        koch_curve(depth-1, headings, length/3., 'F') # F

One thing to note: To prevent our Koch curve from growing in size with each added iteration, we rescale the lengths by a factor of three. This ensures our Koch curve occupies the same horizontal width regardless of the recursion depth.

Next let's defined a general-purpose function that accepts a fractal-generating function as input, sets up the turtle, and executes the drawing. Notice we are keeping track of the turtle's headings over the course of the drawing - you can ignore this for now, but we will make use of those headings later!

In [ ]:
def make_fractal(fractal_fun, depth, length, symbol, # fractal settings
                 position, angle, draw_speed): # turtle settings
  # speed is integer on [1,13]
  
  hideturtle()
  penup() # lift the pen
  setx(position[0])
  sety(position[1])
  setheading(angle)
  pendown() # lower the pen
  showturtle()    

  headings = [angle] # accumulate headings
  speed(draw_speed)
  fractal_fun(depth, headings, length, symbol)
  hideturtle()

  return headings

In [ ]:
fractal_fun = koch_curve
length = W//3
angle = 0

clear()
# plot three different generations of the Koch curve
color(palette['blue'])
h1 = make_fractal(fractal_fun, 1, length, 'F', (W//3, 100), angle, 6)

color(palette['yellow'])
h2 = make_fractal(fractal_fun, 2, length, 'F', (W//3, 200), angle, 8)

color(palette['red'])
h3 = make_fractal(fractal_fun, 3, length, 'F', (W//3, 300), angle, 10)

## Multi-symbol axioms

How can we generate the Koch snowflake using three instances of the Koch curve? We can think of it as a longer axiom with the same alphabet and rules:

1. Alphabet: `F` (variable) `+`, `-` (constants)
2. Axiom: `F--F--F`
3. Rule: `F --> F+F--F+F`

In [ ]:
fractal_fun = koch_curve
length = W//3
angle = 0
depth = 3

clear()
color(palette['blue'])

# Axiom F--F--F
h = make_fractal(fractal_fun, depth, length, 'F', (W//3, H//3), angle, 10)

right(120) # --
h += make_fractal(fractal_fun, depth, length, 'F', (getx(),gety()), getheading(), 10) # continue as part of the same curve

right(120) # --
h += make_fractal(fractal_fun, depth, length, 'F', (getx(),gety()), getheading(), 10) # continue as part of the same curve

### Your turn!

Complete the function below to generate a fractal crystal. The crystal is generated by the following L-system:

1. Alphabet: `F` (variable) `+`, `-` (constants)
2. Axiom: `F+F+F+F`
3. Rule: `F --> FF+F++F+F`

Use a length factor of 1/3 at each iteration (as with the Koch curve), and a rotation of 90 degrees for each left or right turn. The axiom has already been implemented for you in the second cell below. Your code should produce the following pattern:

In [ ]:
#function to create crystal
def crystal(depth, headings, length, symbol='F'):
    if depth == 0:
        forward(length)
        headings += [heading()]
        return
    else:
        # YOUR CODE HERE #
        crystal(depth-1, headings, length/3., 'F')
        crystal(depth-1, headings, length/3., 'F')

        left(90)
        crystal(depth-1, headings, length/3., 'F')

        left(180)
        crystal(depth-1, headings, length/3., 'F')

        left(90)
        crystal(depth-1, headings, length/3., 'F')

In [ ]:
fractal_fun = crystal
length = W//3
angle = 270
depth = 3

clear()
color(palette['red'])

# Axiom F+F+F+F
h = make_fractal(fractal_fun, depth, length, 'F', (2*W//3, 3*H//4), angle, 10)

left(90) # +
h += make_fractal(fractal_fun, depth, length, 'F', (getx(), gety()), getheading(), 10)

left(90) # +
h += make_fractal(fractal_fun, depth, length, 'F', (getx(), gety()), getheading(), 10)

left(90) # +
h += make_fractal(fractal_fun, depth, length, 'F', (getx(), gety()), getheading(), 10)


## Multi-variable L-systems

The L-systems we've seen thus far all had one variable: F. What about multi-variable L-systems?

Consider the following:
1. Alphabet: `F`, `G` (variables) `+`, `-` (constants)
2. Axiom: `F`
3. Rules: `F --> F-G`, `G --> F+G`

The combination of two types of symbols allows us to generate the dragon curve:

In [ ]:
#function to create the dragon curve
def dragon_curve(depth, headings, length, symbol):
    if depth == 0:
        if symbol == 'F': color(palette['red'])
        else: color(palette['blue'])
        forward(length)
        headings += [heading()]
        return
    else:
      dragon_curve(depth-1, headings, length/np.sqrt(2), 'F')
      
      if symbol == 'F': right(90)
      else: left(90)
      dragon_curve(depth-1, headings, length/np.sqrt(2), 'G')

In [ ]:
fractal_fun = dragon_curve
length = W//3

clear()
h = make_fractal(fractal_fun, 8, length, 'F', (W//3, H//2), 0, 12)

### Your turn!

Complete the function below to generate the Sierpinski arrowhead curve. The curve is generated by the following L-system:

1. Alphabet: `F`, `G` (variables) `+`, `-` (constants)
2. Axiom: `F`
3. Rules: `F --> G-F-G`, `G --> F+G+F`

Use a length factor of 1/2 at each iteration, and a rotation of 60 degrees for each left or right turn. The axiom has already been implemented for you in the second cell below. Your code should produce the following pattern:

In [ ]:
#function to create Sierpinski arrowhead curve
def sierpinski_curve(depth, headings, length, symbol):
    if depth == 0:
        if symbol == 'F': color(palette['blue'])
        else: color(palette['yellow'])
        forward(length)
        headings += [heading()]
        return
    else:
        # YOUR CODE HERE
        if symbol == 'F':
            sierpinski_curve(depth-1, headings, length/2., 'G')
            right(60)
            sierpinski_curve(depth-1, headings, length/2., 'F')
            right(60)
            sierpinski_curve(depth-1, headings, length/2., 'G')

        else:
            sierpinski_curve(depth-1, headings, length/2., 'F')
            left(60)
            sierpinski_curve(depth-1, headings, length/2., 'G')
            left(60)
            sierpinski_curve(depth-1, headings, length/2., 'F')

In [ ]:
fractal_fun = sierpinski_curve
length = W//4
angle = 0

clear()
# three different generations of the Sierpinski triangle
h1 = make_fractal(fractal_fun, 3, length, 'F', (W//8,3*H//4), 300, 8)
h2 = make_fractal(fractal_fun, 4, length, 'F', (3*W//8,H//2), 0, 10)
h3 = make_fractal(fractal_fun, 5, length, 'F', (5*W//8,3*H//4), 300, 12)

## Push-Pop Operations

The final twist to L-systems we'll introduce here are push-pop operations, denoted by brackets `[`,`]`. These indicate the following:
- `[` : Save current position and heading (push)
- `]` : Reset to most recent saved position and heading (pop)

Typically, we'll want to lift our pen when resetting to a previous position. Let's define two helper functions for the push-pop operations:

In [ ]:
def push():
    x = getx()
    y = gety()
    h = getheading()
    return x, y, h

def pop(x, y, h):
    penup()
    setx(x)
    sety(y)
    setheading(h)
    pendown()

Push and pop are particularly useful for creating tree-like fractals, which have a lot of branching structures. Let's look at one example below.

1. Alphabet: `F` (variable) `+`, `-`, `[`, `]` (constants)
2. Axiom: `F`
3. Rule: `F --> [+F]F[-F]F`

Rotations here correspond to 30 degrees, and the length factor is 1.

In [ ]:
#function to create fractal tree
def fractal_tree(depth, headings, length, symbol):
    if depth == 0:
        forward(length)
        headings += [heading()]
        return
    else:
        x, y, h = push() # save position and heading
        left(30)
        fractal_tree(depth-1, headings, length, 'F')

        pop(x, y, h) # reset to saved position and heading
        fractal_tree(depth-1, headings, length, 'F')

        x, y, h = push() # save position and heading
        right(30)
        fractal_tree(depth-1, headings, length, 'F')

        pop(x, y, h) # reset to saved position and heading
        fractal_tree(depth-1, headings, length, 'F')

In [ ]:
fractal_fun = fractal_tree
length = 10
angle = 270

clear()
color(palette['green'])

# four generations of the fractal tree
h1 = make_fractal(fractal_fun, 1, length, 'F', (W//5,3*H//4), angle, 6)
h2 = make_fractal(fractal_fun, 2, length, 'F', (2*W//5,3*H//4), angle, 8)
h3 = make_fractal(fractal_fun, 3, length, 'F', (3*W//5,3*H//4), angle, 10)
h4 = make_fractal(fractal_fun, 4, length, 'F', (4*W//5,3*H//4), angle, 12)

L-systems provide us with broad flexibility to come up with unique patterns of our own through different combinations of variables, turns, and push-pop operations. A more complex, beautiful leaf example comes from [Paul Bourke](http://paulbourke.net/fractals/lsys/):


1. Alphabet: `F`, `A`, `B`, `X`, `Y` (variables) `+`, `-`, `[`, `]` (constants)
2. Axiom: `A`
3. Rules: `F --> F`, `A --> F[+X]FB`, `B --> F[-Y]FA`, `X --> A`, `Y --> B`

Note: For only the first rule `F --> F`, the subsequent segment length is multiplied by a factor of 1.36. For all other rules, segment length is left unchanged. `+` and `-` correspond to 45 degree rotations.

In [ ]:
#function to create Paul Bourke's leaf
def bourke_leaf(depth, headings, length, symbol):
    if depth == 0:
        if symbol == 'F': color(palette['green'])
        else: color(palette['red'])

        forward(length)
        headings += [heading()]
        return
    else:
        if symbol == 'F':
            bourke_leaf(depth-1, headings, 1.36*length, 'F')

        elif symbol == 'A':
            bourke_leaf(depth-1, headings, length, 'F')
            x, y, h = push()
            left(45)
            bourke_leaf(depth-1, headings, length, 'X')
            pop(x, y, h)
            bourke_leaf(depth-1, headings, length, 'F')
            bourke_leaf(depth-1, headings, length, 'B')

        elif symbol == 'B':
            bourke_leaf(depth-1, headings, length, 'F')
            x, y, h = push()
            right(45)
            bourke_leaf(depth-1, headings, length, 'Y')
            pop(x, y, h)
            bourke_leaf(depth-1, headings, length, 'F')
            bourke_leaf(depth-1, headings, length, 'A')

        elif symbol == 'X':
            bourke_leaf(depth-1, headings, length, 'A')

        else:
            bourke_leaf(depth-1, headings, length, 'B')


In [ ]:
fractal_fun = bourke_leaf
length = 3
angle = 270

clear()
h = make_fractal(fractal_fun, 10, length, 'A', (W//2,H), angle, 12)

### Next Steps
Try designing your own pattern! For inspiration, you can check out [Paul Bourke's page on L-systems](http://paulbourke.net/fractals/lsys/).

# Fractal Symphony

If we could transform a geometric fractal into a musical composition, what would it sound like? To get started, let's install a few additional packages:

In [ ]:
# installation help from: https://groups.google.com/g/music21list/c/THJNzsgK-pg?pli=1
# To render images of musical notes
print('installing lilypond...')
!apt-get install lilypond > /dev/null

# To convert midi files to wav files and play them
print('installing fluidsynth...')
!apt-get install fluidsynth > /dev/null
!cp /usr/share/sounds/sf2/FluidR3_GM.sf2 ./font.sf2

print('done!')

We'll use the library [music21](https://web.mit.edu/music21/) in order to compose and play music based on our generated fractals. Let's import a few key modules and define helper functions for displaying and playing music.

In [ ]:
from music21 import stream, note, scale, meter, tempo, instrument
from IPython.display import Image, Audio

def show(music):
  display(Image(str(music.write('lily.png'))))

def play(music):
  filename = music.write('mid', fp='audio')
  !fluidsynth -ni font.sf2 $filename -F $filename\.wav -r 16000 > /dev/null
  display(Audio(filename + '.wav'))

The following is a simple demonstration of how music21 works. We can define a note by its pitch (C), octave (4th), and duration (half note), and play it.

In [ ]:
n = note.Note('C4', type='half')
play(n)
show(n)

We can also specify other musical attributes. Here we construct a short score of two parts. Each part is assembled by appending a time signature, tempo, and notes to a "stream" much like writing music. We can also specify a different instrument to use. Each note stream is then assigned to a different part, and the parts are inserted into the same score.

In [ ]:
sc = stream.Score(id='score')
ts = meter.TimeSignature('4/4')
tm = tempo.MetronomeMark('andante')

st1 = stream.Stream(id='stream1')
st2 = stream.Stream(id='stream2')

# add time signature, tempo marking, and notes to first stream
st1.append(ts)
st1.append(tm)
st1.append(note.Note('C4', type="half"))
st1.append(note.Note('C4', type="half"))

# add time signature, tempo marking, and notes to second stream
st2.append(ts)
st2.append(tm)
st2.append(note.Note('E4', type="quarter"))
st2.append(note.Note('F4', type="quarter"))
st2.append(note.Note('E4', type="quarter"))
st2.append(note.Note('F4', type="quarter"))

# create two parts from the two streams
p1 = stream.Part(id='part1')
p1.append(st1.makeMeasures())
p1.insert(0, instrument.Violin())
p2 = stream.Part(id='part2')
p2.append(st2.makeMeasures())

# insert parts into the score
sc.insert(0, p1)
sc.insert(0, p2)
play(sc)
show(sc)

Next, we'll define how we attribute a musical interpretation to our geometric fractals. We've constructed fractals by orienting our turtle at different headings and drawing lines as we go. We'll interpret each change in heading as a change in pitch up or down following a particular scale. The number of steps up or down the scale will correspond to the magnitude of the turn, measured as some multiple of a base rotation angle (e.g. 45 degrees, 60 degrees). First, we define a function that converts our list of headings returned from each fractal simulation to steps relative to a base starting pitch, to be chosen later.

In [ ]:
def headings2steps(headings, base_rotation):
    h = np.array(headings, dtype=int) # headings array
    diff = np.diff(h).astype(int) # sequential changes in heading
    for i in range(len(diff)):
        if np.abs(diff[i]+360) < np.abs(diff[i]): # pick smallest absolute angle
            diff[i] += 360
        if np.abs(diff[i]-360) < np.abs(diff[i]):
            diff[i] -= 360
    steps = np.cumsum(diff / base_rotation) # cumulative sum, so each entry reflects the steps up or down relative to starting pitch
    return steps.astype(int)

As a starting example, let's generate the Koch curve once again, and convert the changes in heading to steps with a base rotation angle of 60 degrees.

In [ ]:
fractal_fun = koch_curve
length = W//3
angle = 0

clear()
color(palette['blue'])
h1 = make_fractal(fractal_fun, 1, length, 'F', (W//3, 100), angle, 6)
color(palette['yellow'])
h2 = make_fractal(fractal_fun, 2, length, 'F', (W//3, 200), angle, 8)
color(palette['red'])
h3 = make_fractal(fractal_fun, 3, length, 'F', (W//3, 300), angle, 10)

s1 = headings2steps(h1, 60)
s2 = headings2steps(h2, 60)
s3 = headings2steps(h3, 60)

In [ ]:
print(h1)
print(s1)

In [ ]:
print(h2)
print(s2)

In [ ]:
print(len(s1),len(s2),len(s3))

Next, we treat each sequence of steps, corresponding to the three different generations of the Koch curve, as a "part" in our "score", with a different starting base pitch and octave. Notice that the length of each part is different, as later generations have more turns. Thus, we select the note durations appropriately so each part has the same overall duration. In this case, the first generation Koch curve will play as whole notes (4 beats in 4/4 time), the second as quarter notes (1 beat), and the third as sixteenth notes (1/4 beat).

In [ ]:
sc = scale.MajorScale('C')
notes = [str(p) for p in sc.getPitches('C1', 'C7')]

ts = meter.TimeSignature('4/4')
tm = tempo.MetronomeMark('andante')
base_pitches = ['C5','E4','G3']   # highest generation to lowest generation
note_durations = [0.25, 1.0, 4.0] # sixteenth note, quarter note, whole note

The following function assembles our steps, time signature, tempo, pitches, and note durations into a complete score:

In [ ]:
def generate_score(parts, scale, notes, base_pitches, time_signature, tempo, note_durations, instruments=None):
    if len(parts) != len(base_pitches) or len(parts) != len(note_durations):
        raise ValueError("Number of parts must equal number of base pitches and note durations.")

    sc = stream.Score(id='score')

    for j,part in enumerate(parts):
        st = stream.Stream(id='stream'+str(j))
        st.append(time_signature)
        st.append(tempo)
        base_index = notes.index(base_pitches[j])
        for i in part:
            n = note.Note(notes[i+base_index])
            n.duration.quarterLength = note_durations[j]
            st.append(n)
        p = stream.Part(id='part'+str(j))
        p.append(st.makeMeasures())
        if instruments is not None:
            p.insert(0, instruments[j])
        sc.insert(0, p)

    return sc

In [ ]:
sc = generate_score([s3,s2,s1], sc, notes, base_pitches, ts, tm, note_durations)
play(sc)
show(sc)

Let's try the Sierpinski arrowhead curve:

In [ ]:
fractal_fun = sierpinski_curve
length = W//4

clear()
# three different generations of the Sierpinski triangle
h1 = make_fractal(fractal_fun, 2, length, 'F', (W//8,3*H//4), 0, 8)
h2 = make_fractal(fractal_fun, 3, length, 'F', (3*W//8,H//2), 300, 10)
h3 = make_fractal(fractal_fun, 4, length, 'F', (5*W//8,3*H//4), 0, 12)

s1 = headings2steps(h1, 60)
s2 = headings2steps(h2, 60)
s3 = headings2steps(h3, 60)

In [ ]:
print(len(s1), len(s2), len(s3))

Since the lengths increase by factors of 3, we'll write this piece in 6/8 time (6 beats per measure, eighth note gets the beat).

In [ ]:
sc = scale.MinorScale('A')
notes = [str(p) for p in sc.getPitches('A1', 'A7')]

ts = meter.TimeSignature('6/8')
tm = tempo.MetronomeMark('allegro')
base_pitches = ['F5','A4','D3']
note_durations = [0.5, 1.5, 4.5]
instruments = [instrument.Guitar(),
               instrument.Guitar(),
               instrument.Guitar()]

In [ ]:
sc = generate_score([s3,s2,s1], sc, notes, base_pitches, ts, tm, note_durations, instruments)

play(sc)
show(sc)

In [ ]:
fractal_fun = fractal_tree
length = 10
angle = 270

clear()
color(palette['green'])
h1 = make_fractal(fractal_fun, 2, length, 'F', (W//4,3*H//4), angle, 12)
h2 = make_fractal(fractal_fun, 3, length, 'F', (W//2,3*H//4), angle, 12)
h3 = make_fractal(fractal_fun, 4, length, 'F', (3*W//4,3*H//4), angle, 12)

s1 = headings2steps(h1, 30)
s2 = headings2steps(h2, 30)
s3 = headings2steps(h3, 30)

In [ ]:
print(len(s1), len(s2), len(s3))

In [ ]:
sc = scale.MajorScale('C')
notes = [str(p) for p in sc.getPitches('C1', 'C7')]

ts = meter.TimeSignature('4/4')
tm = tempo.MetronomeMark('moderato')
base_pitches = ['C6','E4','G3']
note_durations = [0.25, 1.0, 4.0]
instruments = [instrument.Flute(),
               instrument.Oboe(),
               instrument.Bassoon()]

In [ ]:
sc = generate_score([s3,s2,s1], sc, notes, base_pitches, ts, tm, note_durations, instruments)
play(sc)
show(sc)